In [1]:
%load_ext autoreload
%autoreload 2

from train import *
from models import *
from tools import *

from keras import optimizers
from keras.models import load_model
from keras.utils import to_categorical
from keras.callbacks import *
from keras.preprocessing.image import ImageDataGenerator

from PIL import Image
from IPython.display import display

import numpy as np
import os

Using TensorFlow backend.


In [2]:
X_train, y_train = load_data()
X_val, y_val = load_data("val")
X_test, _ = load_data("test")

input_shape = X_train[0].shape
print("Input shape: " + str(input_shape))
print(X_train[0][0][0])

y_train = to_categorical(y_train, 200)
y_val = to_categorical(y_val, 200)

Loading train data

Finished loading train data

Loading val data

Finished loading val data

Loading test data

Finished loading test data

Input shape: (64, 64, 3)
[0.75294118 0.83137255 0.83529412]


In [3]:
labels = get_label_dict()
words = get_word_labels()

test_images = os.listdir("data/tiny-imagenet-200/test/images/")
assert len(X_test) == len(test_images)

Loading labels

Done

Loading words

Done



In [4]:
# Image preprocessing for the data
datagen = ImageDataGenerator(horizontal_flip=True, 
                             shear_range=20., 
                             zoom_range=0.4, 
                             width_shift_range=0.2,
                             height_shift_range=0.2, 
                             channel_shift_range=0.1, 
                             fill_mode="reflect", 
                             preprocessing_function=shuffle_channels)

testgen = ImageDataGenerator()

data = {"train": (X_train, y_train), 
        "val": (X_val, y_val)}

In [5]:
keras.backend.clear_session()

model = AlphaNet(input_shape, "v5")
model_prefix = "models/" + model.name + "/" + model.name
model_path = model_prefix + ".h5"

Model: "AlphaNet_v5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 64, 64, 32)        4736      
_________________________________________________________________
batch_normalization_1 (Batch (None, 64, 64, 32)        128       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 64, 64, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
spatial_dropout2d_1 (Spatial (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 64)        51264     
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 64)        

In [6]:
# Saves the best model based on monitor value between epochs
checkpoint = ModelCheckpoint(model_path, 
                             monitor="val_accuracy", 
                             verbose=1, 
                             save_best_only=True, 
                             period=1)

# Stops training early when no improvement for monitor value
early_stop = EarlyStopping(monitor="val_accuracy", 
                           min_delta=0, 
                           patience=10, 
                           verbose=1)

# Reduces learning rate when monitor value plateaus
reduce_lr = ReduceLROnPlateau(monitor="val_accuracy", 
                              factor=0.316, 
                              patience=5, 
                              verbose=1)

# Logs training data to CSV
restore = False # Whether or not we want to restore weights and continue training from a previous session
csv_log = CSVLogger(model_prefix + ".csv", separator=',', append=restore)

cb_list = [checkpoint, early_stop, reduce_lr, csv_log]

In [7]:
optim = optimizers.Adam(learning_rate=0.001)

history = train(model_path=model_path, 
                restore=restore, 
                epochs=300,
                model=model, 
                optim=optim, 
                datagen=datagen, 
                testgen=testgen, 
                data=data, 
                cb_list=cb_list, 
                batch_size=32)

Epoch 1/300
 - 130s - loss: 5.6325 - accuracy: 0.0075 - top3_accuracy: 0.0216 - top5_accuracy: 0.0342 - val_loss: 5.0071 - val_accuracy: 0.0269 - val_top3_accuracy: 0.0646 - val_top5_accuracy: 0.0935

Epoch 00001: val_accuracy improved from -inf to 0.02690, saving model to models/AlphaNet_v5/AlphaNet_v5.h5
Epoch 2/300
 - 95s - loss: 5.4246 - accuracy: 0.0139 - top3_accuracy: 0.0362 - top5_accuracy: 0.0560 - val_loss: 4.9464 - val_accuracy: 0.0370 - val_top3_accuracy: 0.0884 - val_top5_accuracy: 0.1266

Epoch 00002: val_accuracy improved from 0.02690 to 0.03700, saving model to models/AlphaNet_v5/AlphaNet_v5.h5
Epoch 3/300
 - 94s - loss: 5.3029 - accuracy: 0.0179 - top3_accuracy: 0.0471 - top5_accuracy: 0.0718 - val_loss: 4.4860 - val_accuracy: 0.0473 - val_top3_accuracy: 0.1076 - val_top5_accuracy: 0.1553

Epoch 00003: val_accuracy improved from 0.03700 to 0.04730, saving model to models/AlphaNet_v5/AlphaNet_v5.h5
Epoch 4/300
 - 95s - loss: 5.2190 - accuracy: 0.0224 - top3_accuracy: 0.

KeyboardInterrupt: 

In [ ]:
show_history(history)

In [ ]:
# This loads the model and makes predictions on the test set
custom_metrics = {
    "top3_accuracy": top3_acc,
    "top5_accuracy": top5_acc
}
model = load_model(model_path, custom_objects=custom_metrics)
output = model.predict(X_test)
results = []

for result in output:
    results.append(labels[np.argmax(result)])
    
assert len(results) == len(X_test)
print("Test images predicted:", len(results))

In [ ]:
# This writes the test set predictions to file
with open(model_prefix + " Predictions.txt", "w") as test_file:
    for i in range(len(results)):
        test_file.write(test_images[i] + " " + results[i] + "\n")
    print("Predictions saved at", test_file.name)

In [ ]:
# This shows some test set images and the model's predictions on them
display_dim = 128
for _ in range(10):
    index = np.random.randint(len(X_test))
    word_label = words[results[index]]
    plt.plot()
    plt.title(word_label)
    plt.imshow(X_test[index])
    plt.show()

In [ ]:
# This saves an HTML version of the notebook for later viewing
# NOTE: Make sure to check if the html was completely saved, sometimes it doesn't
from nbconvert import HTMLExporter
import codecs
import nbformat
exporter = HTMLExporter()
output_notebook = nbformat.read("Training.ipynb", as_version=nbformat.NO_CONVERT)
output, resources = exporter.from_notebook_node(output_notebook)
codecs.open(model_prefix + ".html", "w").write(output)